# Listas de Acuerdos

Reporte de Listas de Acuerdos.

In [1]:
from datetime import date

# Pandas, matplotlib, seaborn
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# SQLAlchemy
from plataforma_web.app import create_app
from plataforma_web.extensions import db
app = create_app()
db.app = app

# Plataforma Web
from plataforma_web.blueprints.distritos.models import Distrito
from plataforma_web.blueprints.autoridades.models import Autoridad
from plataforma_web.blueprints.listas_de_acuerdos.models import ListaDeAcuerdo

In [2]:
# Distritos Judiciales
distritos_select = db.session.query(Distrito.id, Distrito.nombre).\
    filter(Distrito.es_distrito_judicial == True).\
    filter(Distrito.estatus == "A").statement
distritos = pd.read_sql_query(sql=distritos_select, con=db.engine)
distritos.head(8)

,id,nombre
0,1,Distrito Judicial de Acuña
1,2,Distrito Judicial de Monclova
2,3,Distrito Judicial de Parras de la Fuente
3,4,Distrito Judicial de Río Grande (Piedras Negras)
4,5,Distrito Judicial de Sabinas
5,6,Distrito Judicial de Saltillo
6,7,Distrito Judicial de San Pedro de las Colonias
7,8,Distrito Judicial de Torreón


In [ ]:
# Distrito Judicial a estudiar
distrito = Distrito.query.get(6)  # 6: Saltillo
print(distrito.id, distrito.nombre)

In [ ]:
# Juzgados
autoridades_select = db.session.query(Autoridad.id, Autoridad.clave, Autoridad.descripcion).\
    filter(Autoridad.distrito_id == distrito.id).\
    filter(Autoridad.es_jurisdiccional == True).\
    filter(Autoridad.es_notaria == False).\
    filter(Autoridad.estatus == "A").statement
autoridades = pd.read_sql_query(sql=autoridades_select, con=db.engine)
autoridades.head(4)

In [ ]:
# Constantes
DESDE = date(year=2021, month=8, day=1)
HASTA = date(year=2021, month=8, day=31)

# Listas de Acuerdos
listas_de_acuerdos_select = db.session.query(ListaDeAcuerdo.id, Autoridad.clave, ListaDeAcuerdo.fecha, ListaDeAcuerdo.archivo, ListaDeAcuerdo.creado).\
    join(Autoridad).\
    filter(ListaDeAcuerdo.autoridad_id.in_(autoridades.id)).\
    filter(ListaDeAcuerdo.fecha >= DESDE).\
    filter(ListaDeAcuerdo.fecha <= HASTA).\
    filter(ListaDeAcuerdo.estatus == "A").statement
listas_de_acuerdos = pd.read_sql_query(sql=listas_de_acuerdos_select, con=db.engine)

# Convertir fecha a datetime
listas_de_acuerdos['fecha'] = pd.to_datetime(listas_de_acuerdos['fecha'])
listas_de_acuerdos.info()

In [ ]:
# Contar listas de acuerdos entregadas por juzgado
listas_de_acuerdos.clave.value_counts()

In [ ]:
# Elaborar matriz juzgados/fechas